In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from IPython.display import display, Markdown, Latex
import pandas as pd
from uncertainties import ufloat
from uncertainties import unumpy as unp
from scipy.optimize import curve_fit
from scipy import constants as const 

# PW8

In diesem Experiment betrachten wird verschiedene Aufbauten von einem Ohm‘schen Widerstand R, einem Kondensator mit Kapazität C und einer Spule mit Induktivität L. Die drei oben genannten Objekte sind stets in Serie geschalten und an einer Wechselspannungsquelle angeschlossen.

Da es sich um eine Wechselspannungsquelle handelt, muss man bei der Auswertung der Daten anders vorgehen als bei der Gleichspannungsquelle. Der Ohm’sche Widerstand R kann analog zum Fall mit Gleichspannung behandelt werden also mittels U0/I0. Für die Beträge der komplexen Impedanzen, also die reellen Wechselstromwiderstände, des Kondensators und der Spule XC und XL herrschen jedoch stets Frequenzabhängigkeiten. Diese lauten wie folgt:

$$
X_C = \frac{1}{\omega C}
$$
bzw.
$$
X_L = \omega L
$$

Das Experiment fordert nun, die Größen C, also die Kapazität des Kondensators in Farad, und L, also die Induktivität der Spule in Henry, zu bestimmen. Dabei geht man wie folgt vor: man misst zunächst den Ohm’schen Widerstand R, der in Serienschaltung mit zunächst nur dem Kondensator und dann nur mit der Spule liegt. Durch Messung von UX und UR, X steht hier im ersten Experiment für C und im zweiten für L, in Abhängigkeit von der Frequenz ν, lässt sich mittels folgender Gleichung der Betrag der Impedanzen berechnen: 
$$
|X| = R \cdot \frac{|U_X|}{|U_R|}
$$

Da $\omega = 2 \pi \nu$ lässt sich nun auch ein Graph $X(\omega)$ erstellen. Fertigt man nun auch einen Fit für die beiden Graphen an lassen sich C und L bestimmen. 

Zum Schluss baut man einen Serienschwingkreis mit R, C und L. Durch Messung von U0 und UC als Funktion der Kreisfrequenz ω und Umformung der Gleichung 

$$
U_C = U_0 \frac{\omega_0^2}{\sqrt{(\omega_0^2 - \omega^2)^2 + 4 \delta^2 \omega^2}}
$$
nach $\frac{U_C}{U_0}$

lässt sich, erneut durch einen Fit, δ und ω0 bestimmen. $\delta = \frac{R}{2 L}$ ist eine Dämpfungskonstante und $\omega_0 = \frac{1}{\sqrt{L C}}$ beschreibt die Eigenfrequenz, bei welcher sich die komplexen Impedanzen des Kondensators und der Spule aufheben. Somit ist hier, bei $\omega = \omega_0$, der Scheinwiderstand minimal.

In [ ]:
# 1. 

R = ufloat+( , ) #widerstand

U_l_nom = np.array([]) # spannungen an der spule
u_U_l = np.array([])
U_l = unp.uarry(U_l_nom, u_U_l)

U_c_nom = np.array([]) # spannungen am kondensator
u_U_c = np.array([])
U_c = unp.uarray(U_c_nom, u_U_c)

U_R_l_nom = np.array([])
u_U_R_l = np.array([])
U_R_l = unp.uarray(U_R_l_nom, u_U_R_l)

U_R_c_nom = np.array([])
u_U_R_c = np.array([])
U_R_c = unp.uarray(U_R_c_nom, u_U_R_c)

nü = unp.uarray([500, 700, 900, 1100, 1300, 1500, 1700, 1900, 2100, 2300, 2500], (0.1 + 5*10**(-5))) # in Hertz 
omega = 2 * np.pi * nü
omega_nom = unp.nominal_values(omega)

Xc = R * (U_c/U_R_c)
Xc_nom = unp.nominal_values(Xc)

Xl = R * (U_l/U_R_l)
Xl_nom = unp.nominal_values(Xl)
u_Xl = unp.std_devs(Xl)

def fit_c(omega_nom, C):
    return 1/(omega_nom*C)

# fit  kondensator durchführen
params, cov = curve_fit(fit_c, omega_nom, Xc_nom)

C_fit = params[0]
u_C = np.sqrt(cov[0,0])

chi2c = np.sum(((Xc_nom-C_fit)/u_Xl)**2)

SS_res_c = np.sum((Xc_nom - C_fit)**2)
SS_tot_c = np.sum((Xc_nom - np.mean(Xc_nom))**2)
R2_c = 1 - (SS_res_c / SS_tot_c)

# fit spule

def fit_l(k, omega_nom, d):
    return k * omega_nom + d

popt, pcov = curve_fit(fit_l, omega_nom, Xl_nom, sigma=u_Xl, absolute_sigma=True)
k, d = popt
dk, dd = np.sqrt(np.diag(pcov)) 

omega_fit = np.linspace(omega_nom.min(), omega_nom.max(), 200)
Xl_fit = fit_l(k, omega_fit, d)

chi2l = np.sum(((Xl_nom-Xl_fit)/u_U_l)**2)

SS_res_l = np.sum((Xl_nom - Xl_fit)**2)
SS_tot_l = np.sum((Xl_nom - np.mean(Xl_nom))**2)
R2_l = 1 - (SS_res_l / SS_tot_l)

# plot
plt.figure()
plt.scatter(omega_nom, Xl_nom, "o", label="Messdaten X_L", color='blue')
plt.scatter(omega_nom, Xc_nom, "o", label="Messdaten X_C", color='red')
plt.plot(omega_fit, Xl_fit, label="Linearer Fit X_L = kx +d", color='green') # beschriftung !!
plt.plot(omega_fit, C_fit, label="Fit X_C = 1/(x*C)", color='yellow')
plt.xlabel("Kreisfrequenz Omega [rad/s]") # omega!!!!
plt.ylabel("Impedanz [Ohm]") # großes omega
plt.legend()
plt.grid(True)
plt.show()

print(f"Die Kapazität C des Kondensators beträgt {C} Farad, die Induktivität der Spule liegt bei {k} Henry.")
print(f"X^2 Kondensator: {chi2c}, R^2 Kondensator: {R2_c}")
print(f"X^2 Spule: {chi2l}, R^2 Spule: {R2_l}")




SyntaxError: invalid syntax (200265877.py, line 3)

In [ ]:
# schwingkreis
U_0_nom = np.array([])
u_U_0 = np.array([])
U_0 = unp.uarray(U_0_nom, u_U_0)

U_s_nom = np.array([])
u_U_s = np.array([])
U_s = unp.uarray(U_s_nom, u_U_s)

# modellfunktion
def modell(omega_nom, omega0, delta):
    return (omega0**2)/np.sqrt((omega0**2 - omega_nom**2)**2 + 4*(delta**s)*omega_nom**2)

y_data = U_s_nom/U_0_nom

# fit durchführen
popt, pcov = curve_fit(modell, omega_nom, y_data, p0=[1000,1])

omega0_fit, delta_fit = popt

omega_fit = np.linspace(min(omega_nom), max(omega_nom), 500)
y_fit = modell(omega_fit, omega0_fit, delta_fit)

chi2s = np.sum(((U_s_nom - y_fit)/u_U_s)**2)
SS_res_s = np.sum((U_s_nom - y_fit)**2)
SS_tot_s = np.sum((U_s_nom - np.mean(ydata))**2)
R2_s = 1 - (SS_res_s / SS_tot_s) 

# plot

plt.scatter(omega_nom, y_data, "o", color='red')
plt.plot(omega_fit, y_fit, label="Fit", color='blue')
plt.xlabel("Kreisfrequenz omega [rad/s]")
plt.ylabel("Spannungsverhältnis U0/UC")
plt.legend()
plt.grid(True)
plt.show()

# lastenwiderstand R berechnen
r = delta_fit * 2 * k

print(f"Die Eigenfrequenz Omega_0 beträgt {omega0_fit} rad/s und die Verschiebung {delta_fit} rad/s.")
print(f"X^2: {chi2s}, R^2: {R2_s}")
print(f"Aus dem Fit lässt sich der Lastenwiderstand als {r} Ohm bestimmen.")
